<a href="https://colab.research.google.com/github/VK00493839/Data_Science/blob/master/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.version)

In [0]:
string = tf.Variable("this is a string", tf.string)
number = tf.Variable(13,tf.int16)
floating = tf.Variable(1.03,tf.float64)

In [0]:
rank1_tensor = tf.Variable(["test1"], tf.string)
rank2_tensor = tf.Variable([["test1",'OK','Hello'], ["test2",'OK','Hello']], tf.string)

In [0]:
tf.rank(rank1_tensor)

In [0]:
rank2_tensor.shape

In [0]:
tensor1 = tf.ones([2,2,3])
tensor2 = tf.reshape(tensor1, [3,2,2])
tensor3 = tf.reshape(tensor1, [6,2])

In [0]:
tensor3

#Regression

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from six.moves import urllib
from IPython.display import clear_output

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [0]:
print(tf.version)

In [0]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [0]:
dftrain.head()

In [0]:
print(dftrain.loc[0], y_train.loc[0])

In [0]:
dftrain.describe()

In [0]:
dftrain.shape

In [0]:
dftrain.age.hist(bins=30)

In [0]:
dftrain.sex.value_counts().plot(kind='barh')

In [0]:
dftrain['class'].value_counts().plot(kind='barh')

In [0]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

In [0]:
dfeval.shape

In [0]:
CATEGORICAL_COLUMNS = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERICAL_COLUMNS = ['age','fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERICAL_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

print(feature_columns)

Training prcoess with epochs with batch size of 32. If we have 10 epochs our model will see the same dataset 10 times. since we need to feed our data in batches and multiple times we need to create something called input function. The input function simply defines how our dataset will be converted into branches at each epoch.

In [0]:
def make_input_function(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [0]:
train_input_fn = make_input_function(dftrain, y_train)
eval_input_fn = make_input_function(dfeval, y_eval, num_epochs=1, shuffle=False)

Creating the Model

In [0]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])
print(result)

In [0]:
result = list(linear_est.predict(eval_input_fn))
print(result[10])
print(result[10]['probabilities']) # ['prob of survived','prob of not survived'] of record 10 in eval_dataset

#Classfication

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [0]:
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Viginica']

In [0]:
train_path = tf.keras.utils.get_file('iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [0]:
train.head()

In [0]:
train.shape

In [0]:
test.shape

In [0]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

Input function

In [0]:
# Input function
def input_fn(features, labels, training=True, batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  # shuffle and repeat if you are in training mode
  if training:
    ds = ds.shuffle(1000).repeat()
  return ds.batch(batch_size)


# Feature Columns
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

# Building the model

# 1) DNNClassifier(Deep Neural Network)
# 2) LinearClassifier
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30,10],n_classes=3)

classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)

In [0]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}'.format(**eval_result))

# Making Predictions

In [0]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}

print('Please type numeric values as prompted')
for feature in features:
  valid = True
  while valid:
    val = input(feature + ': ')
    if not val.isdigit():
      valid = False
  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

#Clustering

Basic steps for Algorithm K-Means

1) Randomly pick K points to place K centroids
2) Assign all of the data points to centroids by distance. The closest centroid to a point is the one it is assigned to
3) Average of all the points belonging to each centroid to find the middle of those clusters (center of mass). Place the corresponding centroids into that position.
4) Reassign every point once again to the closest centroid.
5) Repeat steps 3-4 until no point chnages with centroid it belongs to.

#Hidden Markov Models

The Hidden Markov Model is a finite set of states each of which is associated with a (generally multi-dimensional) probability distribution. Transistions among the states are governerd by a set of probabilities called Transition Probabilities.

A hidden markov model works with probabilities to predict future events or states. In this section we will learn how to create a hidden markov model that can predict the weather.

#States:
These states could be something like 'warm' and 'cold', 'high' and 'low', 'red','green' and 'blue'. These states are hidden with the model we do not see them directly.

#Observations:
Each state has a particular outcome or observation associated with it based on a probability distribution. Eg: On a hot day Vinay has a 80% chance of being happy and 20% chance of being sad.

#Transitions:
Each state will have a probability defining the likelyhood of transitioning to a different state. Eg: a cold day has a 30% chance of being followed by a hot day and 70% chance of being followed by another cold day.

To create a hidden markov model we need:
a) states
b) observation distribution
c) Transition distribution

https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel

In [0]:
import tensorflow_probability as tfp
import tensorflow as tf

1) cold day endoded as 0 and hot day as 1
2) First day in our sequence has an 80% chance of being cold
3) Cold day has 30% chance of being followed by a hot day
4) Hot day has 20% chance of being followed by a cold day
5) On each day the temperature is normally distributed with mean and standard devaiation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.

In [0]:
tfd = tfp.distributions

initial_distribution = tfd.Categorical(probs=[0.8, 0.2]) # refer point 2

transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]]) # refer point 3 a4d 4

observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.]) # refer point 5

In [0]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

In [0]:
# The expected temperatures for each day are given by:

mean = model.mean()  # shape [7], elements approach 9.0

# The log pdf of a week of temperature 0 is:

print(model.log_prob(tf.zeros(shape=[7])))

with tf.compat.v1.Session() as sess:
  print(mean.numpy()) # print tempeartures for 7 days

#Neural Networks with TensorFlow

N = F(Summation(w*x)+b)
For each layer one bias will be there. Function is any activation function.

If we apply sigmoid activation function at output and previous hidden layer has two neurons then: 
Output=sigmoid(N1*W1 + N2*W2 + b)

Activation functions:
a) RELU (Rectified Linear Unit) (all -ve values treated as 0 and for +ve values it took the same number)
b) Tanh (Hyperbolic Tangent) (compress values between -1 to 1)
c) Sigmoid (compress values between 0 to 1)

# Loss/Cost function
As we know our neural network feeds information through the layers until it eventually reaches an output layer. This layer contains the results that we look at to determine the prediction from your network. In the training phase it is likely your network will make many mistakes and poor predictions. In fact at the start of training your network doesn't know anything(it has random weights and biases).

We need some way of evaluating if  the network is doing well and how well it is doing. For your training data we have the features (input) and the labels (expected output) because of this we can compare the output from your network to the expected output. Based on the difference between these values we can determine if your network has done a good job or poor job. If the network has done a good job we'll make minor changes to the weights and biases if it has done a poor job your changes may be drastic. 

So this is where the cost/loss function comes in. These functions responsible for determining how well the network did. We pass it to the output and the expected output and it returns to us some value representing the cost/loss of the network. This effectively make the networks job to optimize this cost function, trying to make it as low as possible.

Some common loss/cost functions:
a) Mean Squared Error
b) Mean Absolute Error
c) Hinge Loss

# Gradient Descent
Gradient descent and back propagation are closely related. Gradient Descent is the aalgorithm used to find the optimal parameters (weights and biases) for our network. While back propagation is the process of calculating the gradinet that is used in the gradient descent step.

# Optmizer
You may sometimes see the term optmizer/optimization function. This is simply the function that implements back propagation algorithm described above. Here's a list of few common ones:
a) Gradinet Descent
b) Stochastic Gradient Descent
c) Mini-Batch Gradient Descent
d) Momentum
e) Nesterov Accelerated Gradient

https://medium.com/@sdoshi579/optimizers-for-training-neural-network-59450d71caf6

In [0]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

# MNIST Fashion Dataset
This dataset includes 60,000 images for training and 10,000 for validation

In [0]:
fashion_minist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_minist.load_data()

In [0]:
train_images.shape # 60k images that are made up of (28 X 28) = 784 pixels
type(train_images)

In [0]:
train_images[0,23,23]

Our pixel values are between 0 and 255, 0 being black and 255 being white. This means we have grayscale image as there are no color channels.

In [0]:
train_labels[:10]

Our labels are intergers ranging from 0 - 9. Each integer represents a specific article of clothing. We'll create an array of label names indicate which is which

In [0]:
class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Data PreProcessing

The last step before creating our model is to preprocess our data. This simply means applying some prior transformations to your data before feeding it in the model. In this case we'll simply scale all of our greyscale pixel values (0-255) to be between 0 and 1. We can do this by dividing each value in the training and testing by 255.0. We do this because smaller values will make it easier for the model to process your values.

In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0

#Building the model

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)), # input layer (1)
    keras.layers.Dense(128,activation='relu'), # hidden layer (2)
    keras.layers.Dense(10,activation='softmax') # output layer (3)
])

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs=10)

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=1)
print('Test Accuracy: ',test_acc) # overfitting beacause test acc less than train acc(tune parameters to overcome this problem) 

#Making Predictions

In [0]:
predictions = model.predict(test_images) # (60000,28,28)
print(predictions,'\n')
print(predictions[0],'\n')
print(np.argmax(predictions[0]))

In [0]:
print(class_names[np.argmax(predictions[0])]) # predicting the same items
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

# Verifying predictions

In [0]:
COLOR = 'white'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR

def predict(model, image, correct_label):
  class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
  prediction = model.predict(np.array([image]))
  predicted_class = class_names[np.argmax(prediction)]

  show_image(image, class_names[correct_label], predicted_class)

def show_image(img, label, guess):
  plt.figure()
  plt.title("Expected: " + label)
  print("Expected: " + label)
  print("Guess: " + guess)
  plt.xlabel("Guess: " + guess)
  plt.imshow(img, cmap=plt.cm.binary)
  plt.colorbar()
  plt.grid(False)
  plt.show()

def get_number():
  while True:
    num = input("pick a number: ")
    if num.isdigit():
      num = int(num)
      if 0 <= num <= 1000:
        return int(num)
      else:
        print("Try Agian..")


num = get_number()
image = test_images[num]
label = test_labels[num]
predict(model, image, label)

# Deep Computer Vision
In this guide we will learn how to perform image classification and object detection/recognition using Deep Computer Vision with something called a Convolutional Neural Network.

The goal of our CNN will be to classify and detect images or specific objects from within the image. We'll be using image data as our features and a label for those images as our label or output.

We already know how Neural Networks work so we can skip through the basics and move right into the following concepts:
a) Image Data
b) Convolutional Layer
c) Pooling Layer
d) CNN Architectures

The main difference we are about to see in these types of neural networks are the layers that make them up.

# Image Data

So far we have dealt with straight forward data that has 1 or 2 dimesnions. Now we are about to deal with Image data made up of 3 dimensions. These 3 dimensions are:
a) Image Height
b) Image Width
c) Color channels

The number of color channels represents the depth of an image corelates to the colors used in it. For eg., image with 3 channels made up of rgb(red, green, blue) pixels. So for each channel we have 3 numeric values in the range of 0-255 that define its color. For an image of color depth 1 we would likely have a greyscale image one value defining each pixel, again in the range of 0-255.

# Convolutional Neural Network

Each CNN is made up of one or many convolutional layers. These layers are different than the dense layers we have seen previously. Their goal is to find the patterns from within images that can be used to classify the image or parts of it. But this may sound familiar to what our densely connected Neural Network in the previuos section was doing.


The fundamental difference between a dense layer and a convolutional layer is that the dense layer detect patterns globally while convolutional layer detects patterns locally. when we have a densely connected layer each node in that layer sees all the data from the previous layer. This means that this layer is looking ALL of the information and is only capable of analyzing the data in a global capacity. Our convolutional layer however will not be densely connected, this means it can detect local patterns using part of the input data to that layer.

# Multiple Convolutional Layers

Even the basic example we'll use in this guid will be made up of 3 convolutional layers. These layers work together by increasing complexity and abstraction at each subsequent layer. The first layer might be responsible for picking up edges and short lines, while the second layer will take as input these lines and start forming shapes or polygons. Finally the last layer might take these shapes and detrmine which combinations make up a specific image.

# Feature Maps

This term simply stands for a 3D tensor with two special axes(width and height) and one depth axis. Our convolutional layers take feature maps as their input and return a new feature map that represents the presence of specific filters from the previous feature map. These are what we call response maps.

#Layer Parameters
A convolutional network has 2 key parameters

# Filters

A filter is a m X n pattern of pixels that we are looking for an image. The number of filters in a convolutional layer represents how many patterns each layer is looking for and what the depth of our response map will be. If we are looking for 32 different patterns/filters than our output feature map (aka the response map) will have a depth of 32 layers of depth will be a matrix of some size containing values including if the filter was present at that location or not.

# Sample Size

This isn't really the best term to describe this but each convolutional layer is going to examine n X m blocks of pixels in each imgae. Typically we'll consider 3X3 or 5X5 blocks. In the example above 3X3 'Smaple size'. This size will be the same as the size of our filter. Our layers work by sliding these filters of nXm pixels over every possible position in our image and populating a new feature map/response map indicating whether the filter is present at each location.

# After Layers we are going to pool those filter values

min/max/average are differnet pooling techniques

#https://www.tensorflow.org/tutorials/images/cnn

CIFAR Image Dataset having 60,000 32X32 color images with 6000 images of each class

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [0]:
# Load and split dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

In [0]:
# Lets look at one image
IMG_INDEX = 1

plt.imshow(train_images[IMG_INDEX], cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[IMG_INDEX][0]])
plt.show()

# CNN Architecture

A common architecture for a CNN is stack of Conv2D and MaxPooling2D layers followed by a few densely connected layers. To idea is that the stack of convolutional and maxpooling layers extract the features from the image. Then these features are flattened and fed to densely connected layers that determine the class of an image based on the presence of features.

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

#Layer1
Input shape of our data will be 32,32,3 and we will process 32 filters of size 3x3 over our input data. We will also apply the activation function to the output of each convolutional operation.

#Layer2
This layer will perform max pooling operation using 2x2 samples and a stride of 2

# Other Layers
The next step of layers do very similar things but take as input the feature map from previous layer. They also increase the frequencies from 32 to 64. We can do this as our data shrinks in special dimensions as it passed through the layers, meaning we can afford computationally to ad more depth.

In [0]:
model.summary()

# Adding Dense Layers

In [0]:
model.add(layers.Flatten()) #(4,4,64)=>4*4*64=1024 
model.add(layers.Dense(64, activation='relu')) # adding dense layer 64 with activation function relu
model.add(layers.Dense(10)) # adding dense layer 10

In [0]:
model.summary()

# Training

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=4, 
                    validation_data=(test_images, test_labels))

# Evaluating the model

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

#Working with Small Datasets
IN the situation where you dont have millions of images it is difficult to train a CNN from scratch that performs very well. This is why we will learn about a few techniques we can use to train CNN's on small datasets of just a few thousand images.

# Data Augmentation
To avoid overfitting and create a larger dataset from a similar one we can use a technique called data augmentation. This is simply performing random transformations on our images so that our model can generalize better. These transformations can be things like Compressions, rotations, stretches and even color changes. Fortunately keras can help us to do this.

In [0]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# creates a data generator object that transforms images
datagen = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

# pick an image to transform
test_img = train_images[4]
img = image.img_to_array(test_img) # convert image to numpy array
img = img.reshape((1,) + img.shape) # reshapes image

i=0

for batch in datagen.flow(img, save_prefix='test', save_format='jpeg'):
  plt.figure(i)
  plot = plt.imshow(image.img_to_array(batch[0]))
  i += 1
  if i > 4:
    break

  plt.show()

# Pretrained Models
you would have noticed that the model above takes a few minutes to train in the Notebook and only gives an accuracy of 70%. This is okay but surely there is a way to improve on this.

In this section, we will talk about using a pretrained CNN as apart of our own custom network to improve the accuracy of our model. We know that CNN's alone (with no dense layers) dont do anything other than map the presence of features from our input. This means we can use a pretrained CNN, one trained on millions of images, as the start of our model. This will allow us to have a very good convolutional base before adding our own dense layered classifier at the end. Infact by using this technique, we can train a very good classifier for a relatively small dataset(<10,000 images). This is because the convnet already has a very good idea of what features to look for in an image and can find them very effectively. So if we can determine the presence of features all the rest of the model needs to do is determine which combination of features makes a specific image.

# Fine Tuning
When we employ the technique defined above we'll often want tweak the final layers in our convolutional base to work better for our specific problem. This involves not touching or re-training the earlier layers in our convolutional base but only adjusting the final few. We do this because the first layers in our base are very good at extracting low level features like lines and edges, things that are similar for any kind of image. Where the later layers are better at picking up very specific features like shapes or even eyes. If we adjust the final layers than we can look for only features relevant to our very specific problem.

# Using a Pretrained Model
https://www.tensorflow.org/tutorials/images/transfer_learning

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#Dataset

We will load the cas_vs_dogs dataset from the module tensorflow_datasets

This dataset contains (image, label) pairs where images have different dimensions and 3 color channels

In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# split the data into 80% training, 10% testing, and 10% validation
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [0]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

#Data Preprocessing

In [0]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

Apply this function to each item in the dataset using the map method:

In [0]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Lets have a look at our images

In [0]:
for image, label in train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

Now shuffle and batch the data.

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

Inspect a batch of data:

In [0]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

In [0]:
for img, label in raw_train.take(2):
  print('Original Shape: ', img.shape)

for img, label in train.take(2):
  print('New Shape: ', img.shape)

#Create the base model from the pre-trained convnets
You will create the base model from the MobileNet V2 model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset consisting of 1.4M images and 1000 classes. ImageNet is a research training dataset with a wide variety of categories like jackfruit and syringe. This base of knowledge will help us classify cats and dogs from our specific dataset.

First, you need to pick which layer of MobileNet V2 you will use for feature extraction. The very last classification layer (on "top", as most diagrams of machine learning models go from bottom to top) is not very useful. Instead, you will follow the common practice to depend on the very last layer before the flatten operation. This layer is called the "bottleneck layer". The bottleneck layer features retain more generality as compared to the final/top layer.

First, instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the include_top=False argument, you load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

This feature extractor converts each 160x160x3 image into a 5x5x1280 block of features. See what it does to the example batch of images:

In [0]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [0]:
base_model.summary()

#Feature extraction

In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor. Additionally, you add a classifier on top of it and train the top-level classifier.

#Freeze the convolutional base

It is important to freeze the convolutional base before you compile and train the model. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's trainable flag to False will freeze all the layers.

In [0]:
base_model.trainable = False

In [0]:
# Let's take a look at the base model architecture
base_model.summary()

#Add our classifier

Now that we have our base layer setup we can add the classfifier. Insted of flattening the feature map of the base kayer we will use a global avergae pooing layer that will average the entire 5x5 area of each 2D feature map and return to us a single 1280 element vector per filter



In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Finally we will add the prediction layer that will be a single dense neuron. We can do this because we only have two classes to predict for.

In [0]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [0]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

#compile the model

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
len(model.trainable_variables)

#Train the model

In [0]:
initial_epochs = 3
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)


In [0]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [0]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

In [0]:
model.save('dogs_vs_cats.h5')
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')